# 5. Model Setup

In [2]:
import pandas as pd

import tfcaidm
from tfcaidm import Jobs
from tfcaidm import Model
from tfcaidm import Dataset

## Setup

1. Get hyperparameters
2. Load a dataset
3. Create a model

**Autoselect GPU (use only on caidm cluster)**

In [3]:
from jarvis.utils.general import gpus
gpus.autoselect()

[ 2021-11-19 13:47:56 ] CUDA_VISIBLE_DEVICES automatically set to: 1           


In [4]:
YAML_PATH = "/home/brandon/tfcaidm-pkg/configs/ymls/xr_pna/pipeline.yml"

### Hyperparameters

In [5]:
# --- Get hyperparameters
runs = Jobs(path=YAML_PATH)

# --- Hyperparameters for N runs
all_hyperparams = runs.get_params()

# ---- Hyperparameters for run #1
hyperparams = all_hyperparams[0]

In [6]:
hyperparams

{'env/path/root': 'exp',
 'env/path/name': 'xr_pna',
 'env/path/client': '/home/brandon/tfcaidm-pkg/configs/ymls/xr_pna/client.yml',
 'model/model': 'unet',
 'model/conv_type': 'conv',
 'model/pool_type': 'conv',
 'model/eblock': 'conv',
 'model/elayer': 1,
 'model/dblock': 'conv',
 'model/depth': 4,
 'model/width': 32,
 'model/width_scaling': 1,
 'model/kernel_size': [3, 3, 3],
 'model/strides': [2, 2, 2],
 'model/bneck': 2,
 'model/branches': 4,
 'model/atrous_rate': 6,
 'model/order': 'rnc',
 'model/norm': 'bnorm',
 'model/activ': 'leaky',
 'model/attn_msk': 'softmax',
 'train/xs/dat': None,
 'train/ys/pna/mask_id': 'msk',
 'train/ys/pna/remove_bg': True,
 'train/ys/pna/mask_weight': 1,
 'train/ys/pna/output_weight': 5,
 'train/ys/pna/head': 'decoder_classifier',
 'train/ys/pna/n_classes': 2,
 'train/ys/pna/loss': 'sce',
 'train/ys/pna/metric': 'dice',
 'train/trainer/seed': 0,
 'train/trainer/n_folds': 1,
 'train/trainer/batch_size': 8,
 'train/trainer/iters': 3000,
 'train/trainer

### Dataset

In [7]:
client = Dataset(hyperparams).get_client(fold=0)

### Model

Model definition:

```json
{
     'model/model': 'unet',
     'model/conv_type': 'conv',
     'model/pool_type': 'conv',
     'model/eblock': 'conv',
     'model/elayer': 1,
     'model/dblock': 'conv',
     'model/depth': 4,
     'model/width': 32,
     'model/width_scaling': 1,
     'model/kernel_size': [3, 3, 3],
     'model/strides': [2, 2, 2],
     'model/bneck': 2,
     'model/branches': 4,
     'model/atrous_rate': 6,
     'model/order': 'rnc',
     'model/norm': 'bnorm',
     'model/activ': 'leaky',
     'model/attn_msk': 'softmax',
}
 ```

In [8]:
from tensorflow.keras import Input

In [9]:
nn = Model(client)
inputs = client.get_inputs(Input)

In [10]:
outputs = nn.build()

(None, 1, 512, 512, 32)
(None, 1, 256, 256, 32)
(None, 1, 128, 128, 32)
(None, 1, 64, 64, 32)
(None, 1, 32, 32, 32)
(None, 1, 64, 64, 32)
(None, 1, 128, 128, 32)
(None, 1, 256, 256, 32)
(None, 1, 512, 512, 32)


In [11]:
outputs.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dat (InputLayer)                [(None, 1, 512, 512, 0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 1, 512, 512,  64          dat[0][0]                        
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 1, 512, 512,  128         conv3d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 1, 512, 512,  0           batch_normalization[0][0]        
______________________________________________________________________________________________

## Structure

### Inputs

Responsible for any input feature transforms.

In [12]:
nn.inputs

{'dat': <KerasTensor: shape=(None, 1, 512, 512, 1) dtype=float32 (created by layer 'dat')>,
 'msk': <KerasTensor: shape=(None, 1, 512, 512, 1) dtype=float32 (created by layer 'msk')>,
 'pna': <KerasTensor: shape=(None, 1, 512, 512, 1) dtype=uint8 (created by layer 'pna')>}

### Hidden

Responsible for the main model architecture.

In [13]:
nn.backbone

<bound method Model.backbone of <tfcaidm.models.model.Model object at 0x7f3640962670>>

### Outputs

Responsible for adding different output heads for depending on the task.

In [14]:
nn.outputs

<bound method Model.outputs of <tfcaidm.models.model.Model object at 0x7f3640962670>>

### Build

Builds a TF model automatically by applying nn.inputs, nn.backbone, nn.outputs in succession.

In [15]:
nn.build

<bound method Model.build of <tfcaidm.models.model.Model object at 0x7f3640962670>>

### Create

Bundles the model and loss together to create the final trainable model.

In [16]:
nn.create

<bound method Model.create of <tfcaidm.models.model.Model object at 0x7f3640962670>>